In [ ]:
def loadGraphFromFile(file_name):

        n_nodes = 0
        n_edges = 0
        n_terminals = 0

        edges = []
        terminals = []

        with open(file_name, 'r') as file:

            for line in file:
                word_list = line.split()

                if(len(word_list) == 0):
                    continue

                if(word_list[0] == 'Nodes'):
                    n_nodes = int(word_list[1])

                if(word_list[0] == 'Edges'):
                    n_edges = int(word_list[1])

                if(word_list[0] == 'Terminals'):
                    n_terminals = int(word_list[1])

                if(word_list[0] == 'E'):
                    edges.append([int(word_list[1]) - 1, int(word_list[2]) - 1, int(word_list[3])])

                if(word_list[0] == 'T'):
                    terminals.append(int(word_list[1]) - 1)
                    
        return [n_nodes, n_edges, n_terminals, edges, terminals]

In [ ]:
class SteinerTree:
    
    def __init__(self, file_name, opt):
        
        initList = loadGraphFromFile(file_name)
        
        self.name = file_name[-7:-4]
        self.opt = opt
        self.numNodes = initList[0]
        self.numEdges = initList[1]
        self.numTerms = initList[2]
        self.edges = initList[3]
        self.terminals = initList[4]

In [ ]:
bGraphList = [
    ('b01.stp', 82),
    ('b02.stp', 83),
    ('b03.stp', 138),
    ('b04.stp', 59),
    ('b05.stp', 61),
    ('b06.stp', 122),
    ('b07.stp', 111),
    ('b08.stp', 104),
    ('b09.stp', 220),
    ('b10.stp', 86),
    ('b11.stp', 88),
    ('b12.stp', 174),
    ('b13.stp', 165),
    ('b14.stp', 235),
    ('b15.stp', 318),
    ('b16.stp', 127),
    ('b17.stp', 131),
    ('b18.stp', 218)
]

In [ ]:
import os

bDataPath = os.path.abspath(os.getcwd()) + '\\..\\data\\B'

print(bDataPath)
filenames = os.listdir(bDataPath)
print(filenames)

bGraphs = []

for st_args in bGraphList:
    graphpath = bDataPath + '\\' + st_args[0]
    graphopt = st_args[1]
    bGraphs.append(SteinerTree(graphpath, graphopt))
    
print(len(bGraphs))
print(bGraphs[9].name)

In [ ]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from scipy.sparse.csgraph import connected_components
from scipy.sparse.csgraph import shortest_path
import math
import numpy as np

In [ ]:
def makeInitialSolution(terminal_list, adj_matrix):
    
    N = len(adj_matrix)
    
    adj_matrix_c = adj_matrix.copy()
    
    #oznacava koji cvorovi ulaze u izbor za stainerovo stablo, ukljucuje terminale
    selected = np.ones(N)
    
    X = csr_matrix(adj_matrix_c.tolist())
    
    Tcsr = minimum_spanning_tree(X)
    
    Tcsr = Tcsr.toarray().astype(int)
    
    #pravim matricu simetricnom
    Tcsr = Tcsr + Tcsr.T - np.diag(np.diag(Tcsr))
    
    #fleg koji oznacava da nema vise jednostepenih cvorova
    has_one_deg = True
    
    while has_one_deg:
        
        i = 0
        #fleg oznacava da je brisan red
        deleted = False
        
        while i < N:
            
            if terminal_list[i]:
                deleted = True #da ne bi stalo zbog toga sto je naiso na terminal
                i = i+1
                continue
                
            if(np.count_nonzero(Tcsr[i]) == 1):
                selected[i] = 0
                Tcsr[i] = 0   # zeroes out row i
                Tcsr[:,i] = 0 # zeroes out column i
                deleted = True
                break
        
            i = i+1
        
        #print("OVDE")
        if not deleted:
            has_one_deg = False
    
    #print("IZASO")
    
    value = sum(sum(Tcsr.toarray().astype(int))) / 2
    
    return (selected, Tcsr, value)

def pathValue(path, adj_matrix):
    
    N = len(path)
    
    value = 0
    
    for i in range(N-1):
        
        value = value + adj_matrix[path[i]][path[i+1]]
        
    return value
        
def makeNeighbourSolution(solution, terminal_list, adj_matrix):
    
    current_solution = np.copy(solution)
    
    #brisemo random granu
    (i, j) = np.nonzero(current_solution)
    num_pairs = len(i)
    
    random_edge = np.radnom.randint(num_pairs)
    
    current_solution[i[radnom_edge]][j[random_edge]] = 0
    
    #odredjujemo komponente povezanosti
    X = csr_matrix(current_solution.tolist())
    
    n_components, labels = connected_components(X, directed=False, return_labels=True)
    
    #trazimo minimalni put izmedju dve komponente povezanosti
    min_path = []
    min_path_value = 10000 #nesto mnogo
    
    for i in len(labels):
        
        if(labels[i] == 0):
            
            for j in len(labels):
                
                if(labels[j] == 1):
                    
                    graph = csr_matrix(adj_matrix)
                    
                    path = shortest_path(graph, [i, j])
                    
                    path_value = pathValue(path, adj_matrix)
                    
                    if(path_value < min_path_value):
                        min_path_value = path_value
                        min_path = path
                    
    new_solution = np.copy(current_solution)
    
    path_len = len(min_path)
    
    for i in range(path_len-1):
        
        row = min_path[i]
        col = min_path[i+1]
        
        new_solution[row][col] = adj_matrix[row][col]
        new_solution[col][row] = adj_matrix[col][row]
        
    
    new_selected, _ = np.nonzero(new_solution)
        
    #fleg koji oznacava da nema vise jednostepenih cvorova
    has_one_deg = True
    
    while has_one_deg:
        
        i = 0
        #fleg oznacava da je brisan red
        deleted = False
        
        while i < N:
            
            if terminal_list[i]:
                deleted = True #da ne bi stalo zbog toga sto je naiso na terminal
                i = i+1
                continue
                
            if(np.count_nonzero(new_solution[i]) == 1):
                new_selected[i] = 0
                new_solution[i] = 0   # zeroes out row i
                new_solution[:,i] = 0 # zeroes out column i
                deleted = True
                break
        
            i = i+1
        
        if not deleted:
            has_one_deg = False
            
    new_value = sum(sum(new_solution.toarray().astype(int))) / 2
    
    return (new_selected, new_solution, new_value)

def simulatedAnnealing(st: SteinerTree):
    
    N = st.numNodes
    edge_number = st.numEdges
    term_number = st.numTerms
    optimal = st.opt
    edges = st.edges
    terminals = st.terminals
    
    terminal_list = np.zeros(N)
    
    for t in terminals:
        terminal_list[t] = 1
    
    adj_matrix = np.zeros((N, N))
    
    for e in edges:
        adj_matrix[e[0]][e[1]] = e[2]
        adj_matrix[e[1]][e[0]] = e[2]
    
    total_value = np.sum(adj_matrix) / 2
    
    MAX_ITER = 10000
    
    #napraviti moguce resenje problema praveci mcst i brisanjem svih cvorova stepena jedan
    #resenje predstavlje matricu povezanosti stainerovog stabla
    best_selected, best_solution, best_value = makeInitialSolution(terminal_list, adj_matrix)
    
    print(best_value)
    
    iteration = 0
    
    func_eval_GA = 0
    fit_GA = []
    
    while iteration < MAX_ITER:
        
        #napraviti novo resemje iz okoline proslog brisanjem random grane
        #i ponovnim povezivanjem minimalnim putem
        current_selected, current_solution, current_value = makeNeighbourSolution(best_solution, terminal_list, adj_matrix)
        
        #provera vrednosti resenja i potencijalna zamena
        if(current_value < best_value):
            
            best_value = current_value
            best_solution = np.copy(current_solution)
            best_selected = np.copy(best_selected)
            
        else:
            
            p = 1.0 / iteration**0.5
            
            q = np.random.rand()
            
            if p > q:
                best_value = current_value
                best_solution = np.copy(current_solution)
                best_selected = np.copy(best_selected)
                
        iteration = iteration + 1
        func_eval_GA = func_eval_GA + 1
        
        if best_value > total_value:
            fit_GA.append(total_value)
        else:
            fit_GA.append(best_value)
            
        if best_value == optimal:
            break
            
        if(iteration % 20 == 0):
            print(global_best_value)
        
    return (func_eval_GA, fit_GA, best_value)

In [ ]:
func_eval_GA, fit_GA, best_value = simulatedAnnealing(bGraphs[0])

In [ ]:
import matplotlib.pyplot as plt

x = np.arange(0, func_eval_GA, 1)
y = fit_GA

fig, ax = plt.subplots()
ax.plot(x, y)

ax.set(xlabel='GA iterations', ylabel='Steiner Tree value',
       title='Genetic algorithm application to b01 dataset')
ax.grid()

ax.spines['bottom'].set_color('white')
ax.spines['top'].set_color('white') 
ax.spines['right'].set_color('white')
ax.spines['left'].set_color('white')
ax.tick_params(axis='x', colors='white')
ax.tick_params(axis='y', colors='white')
ax.yaxis.label.set_color('white')
ax.xaxis.label.set_color('white')
ax.title.set_color('white')

plt.show()